<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Catboost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [14]:
!pip install catboost
!pip install langid

In [29]:
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer 
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from sklearn.utils import shuffle
from catboost import CatBoostClassifier
import langid
from nltk import pos_tag

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [16]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [17]:
df.head(5)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [19]:
df.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Можно отметить высокую степень дисбаланса классов.

Напишем функцию, которая будет возвращать язык переданного ей текста, и применим ее к столбцу с комментариями

In [20]:
def language_detector(row):
  return langid.classify(row['text'])[0]

df['language'] = df.apply(language_detector, axis=1)

In [21]:
df.head()

,Unnamed: 0,text,toxic,language
0,0,Explanation\nWhy the edits made under my usern...,0,en
1,1,D'aww! He matches this background colour I'm s...,0,en
2,2,"Hey man, I'm really not trying to edit war. It...",0,en
3,3,"""\nMore\nI can't make any real suggestions on ...",0,en
4,4,"You, sir, are my hero. Any chance you remember...",0,en


In [22]:
df.language.value_counts()

en    155664
fr       343
de       341
es       288
it       270
       ...  
mk         1
uk         1
ml         1
bg         1
ka         1
Name: language, Length: 85, dtype: int64

Как видно, язык написания абсолютного большинства комментариев определен как англйиский.

Напишем функцию, которая очищаяет и лемматизирует текст.

In [23]:
lemmatizer = WordNetLemmatizer()

In [61]:
def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize_sent(text):
    clear_text = ' '.join(re.sub(r'[^a-zA-Z ]', ' ', text['text']).split())
    return str.lower(' '.join([lemmatizer.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(nltk.word_tokenize(clear_text))]))  

In [62]:
df['lemmas'] = df.apply(lemmatize_sent, axis=1)

In [63]:
df.head()

,Unnamed: 0,text,toxic,language,lemmas
0,0,Explanation\nWhy the edits made under my usern...,0,en,explanation why the edits make under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,en,d aww he match this background colour i m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,en,hey man i m really not try to edit war it s ju...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,en,more i can t make any real suggestion on impro...
4,4,"You, sir, are my hero. Any chance you remember...",0,en,you sir be my hero any chance you remember wha...


In [65]:
features = df['lemmas']
target = df['toxic']

features_train, features, target_train, target = train_test_split(features, target, test_size=.2, random_state=1)
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size=.5, random_state=1)

Определим набор стоп-слов, которые будут исключены при векторизации текста

In [66]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Обучим векторайзер на обучающей выборке, затем преобразуем им обучающую и тестовую выборки

In [67]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

train_features_tf_idf = tf_idf.fit_transform(features_train)
valid_features_tf_idf = tf_idf.transform(features_valid)
test_features_tf_idf = tf_idf.transform(features_test)

## Обучение

### Логистическая регрессия

In [68]:
model = LogisticRegression()
model.fit(train_features_tf_idf, target_train)
predictions_logistic = model.predict(valid_features_tf_idf)
print('F1-мера для модели логистической регрессии составила {:.2f}'.format(
    f1_score(target_valid, predictions_logistic)))

F1-мера для модели логистической регрессии составила 0.74


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ранее мы обратили внимание, что классы в датасете несбалансированы. Попробуем обучить модель со взвешенными классами.

In [70]:
model = LogisticRegression(class_weight='balanced', max_iter=300)
model.fit(train_features_tf_idf, target_train)
predictions_logistic_balanced_weight = model.predict(valid_features_tf_idf)
print('F1-мера для модели логистической регрессии со взвешенными классами составила {:.2f}'.format(
    f1_score(target_valid, predictions_logistic_balanced_weight)))

F1-мера для модели логистической регрессии со взвешенными классами составила 0.73


### Catboost

In [71]:
features_cb = df[['lemmas']]
target_cb = df[['toxic']]

Поделим выборки на обучающую, валидационную и тестовую.

In [72]:
features_cb_train, features_cb_, target_cb_train, target_cb_ = train_test_split(features_cb, target_cb, test_size=.2, random_state=1)
features_cb_valid, features_cb_test, target_cb_valid, target_cb_test = train_test_split(features_cb_, target_cb_, test_size=.5, random_state=1)

In [73]:
%%time
model = CatBoostClassifier(iterations=500, learning_rate=0.03, depth=10)
model.fit(features_cb_train, target_cb_train, 
          eval_set=(features_cb_valid, target_cb_valid),
          text_features=['lemmas'], verbose=50)

0:	learn: 0.6516775	test: 0.6514702	best: 0.6514702 (0)	total: 4.61s	remaining: 38m 18s
50:	learn: 0.1528952	test: 0.1495818	best: 0.1495818 (50)	total: 3m 53s	remaining: 34m 16s
100:	learn: 0.1324935	test: 0.1307348	best: 0.1307348 (100)	total: 7m 43s	remaining: 30m 30s
150:	learn: 0.1249675	test: 0.1243189	best: 0.1243189 (150)	total: 11m 40s	remaining: 26m 58s
200:	learn: 0.1200472	test: 0.1207664	best: 0.1207664 (200)	total: 15m 28s	remaining: 23m 1s
250:	learn: 0.1159756	test: 0.1180268	best: 0.1180268 (250)	total: 19m 13s	remaining: 19m 4s
300:	learn: 0.1126570	test: 0.1161610	best: 0.1161610 (300)	total: 22m 58s	remaining: 15m 11s
350:	learn: 0.1100628	test: 0.1147212	best: 0.1147212 (350)	total: 26m 46s	remaining: 11m 21s
400:	learn: 0.1074270	test: 0.1135209	best: 0.1135209 (400)	total: 30m 27s	remaining: 7m 31s
450:	learn: 0.1057509	test: 0.1129314	best: 0.1129314 (450)	total: 34m 20s	remaining: 3m 43s
499:	learn: 0.1035772	test: 0.1120677	best: 0.1120657 (498)	total: 38m 18s

In [74]:
predictions_cb = model.predict(features_cb_valid)
print('F1-мера для модели CatBoost составила {:.2f}'.format(
    f1_score(target_cb_valid, predictions_cb)))

F1-мера для модели CatBoost составила 0.78


## Выводы

In [76]:
predictions_cb = model.predict(features_cb_test)
print('F1-мера для модели CatBoost составила {:.2f}'.format(
    f1_score(target_cb_test, predictions_cb)))

F1-мера для модели CatBoost составила 0.78


Выполнен анализ работы моделей логистической регрессии и CatBoost на классификации текстовых комментариев. Модель логистической регрессии показала более слабый результат, но нужно отметить высокую скорость ее работы. Модель CatBoost показала хороший результат, но потребовала значительные траты времени и ресурсов на обучение. Помимо этого, преимущество модели CatBoost заключается в том, что она работает непосредственно с текстовыми признаками, без необходимости в их векторизации.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны